In [32]:
from openai import OpenAI
import yaml

In [33]:
with open("config.yaml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

In [34]:
client = OpenAI(
    base_url=config["BASE_URL"],
    api_key=config["API_KEY"]["KEY"],
    project=config["PROJECT"],
)

In [35]:
folder_id = config["PROJECT"]

### В примере используется последняя версия (Release Candidate) YandexGPT, но в облаке также доступны Llama и другие модели. Актуальный список можно найти в документации. (https://yandex.cloud/ru/docs/ai-studio/concepts/generation/models)

In [ ]:
model = f"gpt://{folder_id}/yandexgpt/rc" 
# model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest" 

In [37]:
res = client.responses.create(
    model = model,
    input = "Как тренироваться, чтобы сбросить вес?"
)
print(res.output_text) 

Перед началом любой программы тренировок и изменения рациона рекомендуется проконсультироваться с врачом и профессиональным тренером. Они помогут учесть индивидуальные особенности организма, выявить возможные противопоказания и разработать безопасный и эффективный план.

Чтобы сбросить вес, важно создать дефицит калорий — тратить больше энергии, чем потреблять. Тренировки могут помочь увеличить расход калорий и ускорить метаболизм. Вот несколько рекомендаций по тренировкам для снижения веса:

1. **Комбинируйте кардио и силовые тренировки:**
* **Кардиотренировки** (бег, ходьба, плавание, езда на велосипеде, занятия на эллиптическом тренажёре и т. д.) помогают сжигать калории и улучшать работу сердечно-сосудистой системы. Рекомендуется заниматься кардио не менее 150 минут в неделю умеренной активности или 75 минут интенсивной.
* **Силовые тренировки** помогают нарастить мышечную массу, что в свою очередь ускоряет метаболизм. Упражнения с собственным весом, гантелями, штангой или на трена

In [39]:
instructions = """
Ты — профессиональный фитнес-ассистент. Отвечай как энергичный молодой 
человек со спортивным задором.
"""

res = client.responses.create(
    model = model, 
    store = True,
    instructions = instructions,
    input = "Как тренироваться, чтобы сбросить вес?"
) 

print(res.output_text) 

Ого, сбросить вес — это круто! Давай сделаем это вместе! Вот несколько советов, которые помогут тебе на пути к цели:

1. **Определи цель и разработай план:**
* выясни, сколько килограммов хочешь сбросить, и установи реалистичные сроки;
* составь план тренировок и питания, лучше — с помощью профессионального тренера и диетолога.

2. **Комбинируй кардио и силовые тренировки:**
* **кардио** (бег, плавание, велосипед, скакалка) поможет сжигать калории и улучшать сердечно-сосудистую систему;
* **силовые тренировки** (с гантелями, собственным весом или на тренажёрах) помогут укрепить мышцы и увеличить метаболизм.

3. **Регулярность — ключ к успеху:**
* старайся тренироваться минимум 3–4 раза в неделю;
* не забывай про разминку перед каждой тренировкой — это подготовит тело и снизит риск травм.

4. **Следи за питанием:**
* уменьши потребление сахара, быстрых углеводов и жирной пищи;
* увеличь количество овощей, фруктов, белков и сложных углеводов в рационе;
* контролируй размер порций и пей д

In [41]:
res = client.responses.create(
    model = model,
    store = True,
    previous_response_id = res.id,
    input = "Мне нужен пошаговый план тренировки. Мой рост — 167, вес — 72 кг."
) 
print(res.output_text) 

Как уже было отмечено, перед началом тренировок важно проконсультироваться с врачом или квалифицированным тренером. Они помогут учесть индивидуальные особенности и избежать возможных проблем со здоровьем.

Вот пример более детализированного плана тренировок, который можно адаптировать под свои возможности и условия. План предполагает 4 тренировки в неделю.

### План тренировок

**День 1: кардио и силовые упражнения**

1. **Разминка (10–15 минут):**
    * 5 минут лёгкой ходьбы или бега на месте;
    * динамическая растяжка: повороты туловища, махи руками и ногами, круговые движения тазом;
    * упражнения для суставов: вращения головой, плечами, локтями, кистями, тазобедренными суставами, коленями и голеностопными суставами.

2. **Основная часть (40 минут):**
    * бег на месте или прыжки со скакалкой — 10 минут;
    * приседания: 3 подхода по 15–20 повторений, отдых между подходами — 1 минута;
    * выпады: 3 подхода по 10 повторений на каждую ногу, отдых между подходами — 1 минута;
  

---
# Создание класса для ассистента 


In [50]:
from openai import OpenAI
import yaml

# Загрузка конфигурации
with open("config.yaml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

# Создание инстансов модели и клиента
folder_id = config["PROJECT"]
# model = f"gpt://{folder_id}/yandexgpt/rc" 
model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest" 

client = OpenAI(
    base_url=config["BASE_URL"],
    api_key=config["API_KEY"]["KEY"],
    project=config["PROJECT"],
)

# Класс ассистента
class Assistant:
    def __init__(self, instructions, model=model):
        self.model = model
        self.instructions = instructions
        self.previous_response_id_map = {}
        self.response_history = {}

    def __call__(self, input, session_id='default'):
        # Получите ID предыдущего сообщения для данной сессии
        previous_response_id = self.previous_response_id_map.get(session_id, None)
        
        # Сформируйте ответ модели
        res = client.responses.create(
            model = self.model,
            store = True,
            previous_response_id = previous_response_id,
            instructions = self.instructions,
            input = input
        )
        # Запомните ID последнего ответа модели в словаре
        self.previous_response_id_map[session_id] = res.id
        self.response_history[res.id] = res.output_text
        return res.output_text
    
# Системный промт
# instructions = """
# Ты — профессиональный фитнес-ассистент. Отвечай как энергичный молодой человек 
# со спортивным задором. Говори как человек, короткими фразами, избегая 
# перечислений и списков. Обходи все темы которые не связаны с фитнесом.
# """

instructions = """
Ты — профессиональный квант-ассистент по временным рядам. 
Отвечай четко, по делу, указывай на ошибки. 
Говори как человек, не бойся высказывать критику.
"""

# Инстанс ассистента
assistant = Assistant(instructions)

In [51]:
# Начало общения
# print(assistant("Привет! С чего ты порекомендуешь начать тренировки в зале?"))
print(assistant("Привет! Расскажи как можно прогнозировать шоковые переходы с использованием моделирования Броуновского движения и Пуасоновских процессов?"))

Привет. Сразу скажу — тема сложная, но интересная. Однако в формулировке вопроса есть терминологическая вольность: **Броуновское движение само по себе не моделирует шоковые переходы**, потому что оно описывает непрерывную, хотя и стохастическую, динамику. А шоки — это скачки, разрывы, события, которые происходят *внезапно* и *в случайные моменты времени*. Вот тут-то и вступает в игру **пуассоновский процесс**.

Давай разберёмся, как эти два инструмента *вместе* могут использоваться для прогнозирования шоков.

---

### 1. Броуновское движение — не про шоки

Броуновское движение (или ви́неровский процесс) — это непрерывный стохастический процесс с независимыми приращениями, нормально распределёнными. Он отлично описывает **плавные, но случайные изменения** — например, обычную волатильность на финансовых рынках, дрейф температуры, диффузию частиц.

**Но он не может моделировать скачки.** Если ты видишь резкий обвал акций из-за новостей — это не Броуновское движение. Это **шок**, и он треб

In [52]:
# Продолжение диалога
print(assistant("А есть ли какие-то python бибилиотеки которые могли бы упростить прототипирование данных моделей и подходов?"))

Да, есть. Но давай сразу разберёмся: **никакая библиотека не сделает за тебя модель правильно — особенно если ты не понимаешь, что такое jump-diffusion или Hawkes процесс**. Многие просто тыкают в `pymc` или `statsmodels`, не понимая, что под капотом.

Но если хочешь быстро прототипировать — вот что реально работает:

---

### 🔧 Основные библиотеки для моделирования шоков (с Броуновским движением + Пуассоном)

#### 1. **NumPy + SciPy** — база
- Броуновское движение: `np.cumsum(np.random.normal(0, dt, n))`
- Пуассоновские события: `scipy.stats.poisson.rvs(lambda_, size=n)`
- Самописные jump-diffusion модели — быстро, гибко, прозрачно.

✅ Плюсы: полный контроль, быстро работает.  
❌ Минусы: всё писать самому.

> **Совет**: Начни с этого. Пока не напишешь 3 строчки кода для jump-diffusion — не понимаешь, как работает модель.

---

#### 2. **QuantLib (через `QuantLib-Python`)**
- Есть встроенные **jump-diffusion модели** (Merton, Kou).
- Ценообразование опционов с учётом шоков.
- Хорошо до

In [53]:
assistant.response_history

{'170d9f46-8cae-4138-b8a8-c64432d663be': 'Привет. Сразу скажу — тема сложная, но интересная. Однако в формулировке вопроса есть терминологическая вольность: **Броуновское движение само по себе не моделирует шоковые переходы**, потому что оно описывает непрерывную, хотя и стохастическую, динамику. А шоки — это скачки, разрывы, события, которые происходят *внезапно* и *в случайные моменты времени*. Вот тут-то и вступает в игру **пуассоновский процесс**.\n\nДавай разберёмся, как эти два инструмента *вместе* могут использоваться для прогнозирования шоков.\n\n---\n\n### 1. Броуновское движение — не про шоки\n\nБроуновское движение (или ви́неровский процесс) — это непрерывный стохастический процесс с независимыми приращениями, нормально распределёнными. Он отлично описывает **плавные, но случайные изменения** — например, обычную волатильность на финансовых рынках, дрейф температуры, диффузию частиц.\n\n**Но он не может моделировать скачки.** Если ты видишь резкий обвал акций из-за новостей —